<a href="https://colab.research.google.com/github/Krishgrv/sample-code/blob/master/QAProj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install allennlp


In [0]:
!echo "Zebras are several species of African equids (horse family) united by their distinctive black and white stripes. Their stripes come in different patterns, unique to each individual. They are generally social animals that live in small harems to large herds. Unlike their closest relatives, horses and donkeys, zebras have never been truly domesticated. There are three species of zebras: the plains zebra, the Grévy's zebra and the mountain zebra. The plains zebra and the mountain zebra belong to the subgenus Hippotigris, but Grévy's zebra is the sole species of subgenus Dolichohippus. The latter resembles an ass, to which it is closely related, while the former two are more horse-like. All three belong to the genus Equus, along with other living equids. The unique stripes of zebras make them one of the animals most familiar to people. They occur in a variety of habitats, such as grasslands, savannas, woodlands, thorny scrublands, mountains, and coastal hills. However, various anthropogenic factors have had a severe impact on zebra populations, in particular hunting for skins and habitat destruction. Grévy's zebra and the mountain zebra are endangered. While plains zebras are much more plentiful, one subspecies - the Quagga - became extinct in the late 19th century. Though there is currently a plan, called the Quagga Project, that aims to breed zebras that are phenotypically similar to the Quagga, in a process called breeding back." > /tmp/sample1
!echo "Which Zebras are endangered?" >> /tmp/sample1
!echo "What is the aim of the Quagga Project?" >> /tmp/sample1
!echo "Which animals are some of their closest relatives?" >> /tmp/sample1
!echo "Which are the three species of zebras?" >> /tmp/sample1
!echo "Which subgenus do the plains zebra and the mountain zebra belong to?" >> /tmp/sample1
!echo "subgenus Hippotigris; the plains zebra, the Grévy's zebra and the mountain zebra;horses and donkeys;aims to breed zebras that are phenotypically similar to the quagga; Grévy's zebra and the mountain zebra" >> /tmp/sample1

In [0]:
import re
import sys
from allennlp.predictors.predictor import Predictor
from difflib import get_close_matches

def ReadInputFromFile(filepath):
   print ("Reading input from file...")
   with open(filepath) as fp:
        global inputText
        global questions
        global answers
        global lqns
        global lans

        line = fp.readline()
        cnt = 1
        while line:
             #print("Line {}: {}".format(cnt, line.strip()))
             if (cnt == 1):
                inputText = line
             if (cnt >=2 and cnt <= 6):
                questions += line.rstrip('\r\n')
             if (cnt > 6):
                answers += line
             line = fp.readline()
             cnt += 1

        lqns = questions.split('?')
        lans = answers.split(';')


def LoadPredictor():
    global predictor
    print ("Loading Predictor...")
    predictor = Predictor.from_path("https://s3-us-west-2.amazonaws.com/allennlp/models/bidaf-model-2017.09.15-charpad.tar.gz")

def ProcessInputAndAnswerQuestions():
   global lqns
   global predictor
   global inputText

   print ("Processing to answer questions...")
   qcnt = 1
   for qn in lqns:
    if (len(qn.strip()) != 0):
       print ('========')
       print ("{}. {}".format(qcnt,  qn + '?'))
       print('.........')
       result = predictor.predict(
                 passage=inputText,
                 question=qn
       )
       bestans = result['best_span_str']
       #print (bestans)
       lbestans = get_close_matches(bestans, lans)       
       if (len(lbestans) > 0):
           print ("Ans: {}".format(lbestans[0]))
       else:
           print ("Error: {}".format("Either Invalid question OR No appropriate answer provided!!"))
       qcnt += 1


#if (len(sys.argv) < 2):
#   print ("Usage : python sahajProj.py <Sample input file name>")
#   sys.exit(1)

#print ("Input file : ", sys.argv[1] )

#filepath = sys.argv[1]
filepath = "/tmp/sample1"
inputText = ""
questions = ""
answers = ""

ReadInputFromFile(filepath)
LoadPredictor()
ProcessInputAndAnswerQuestions()



